In [1]:
import pandas as pd
from datetime import datetime
import spacy
import spacy_transformers
from spacy.tokens import DocBin

c:\Users\YGP\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("1.csv", encoding = 'latin-1')
df.head()

,seg,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [3]:
df['text'][1]

'Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .'

In [4]:
train = df.sample(frac = 0.8, random_state = 25)
test = df.drop(train.index)

In [5]:
print(train.shape, test.shape)

(3877, 2) (969, 2)


In [6]:
nlp = spacy.load("en_core_web_trf")

In [7]:
#In spacy version 3.0, we need to create a training dataset in a binary format to train and test the model
print(nlp.pipe_names)

['transformer', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [8]:
train['tuples'] = train.apply((lambda row : (row['text'], row['seg'])), axis = 1)
test['tuples'] = test.apply((lambda row : (row['text'], row['seg'])), axis = 1)
train = train['tuples'].tolist()
test = test['tuples'].tolist()

In [9]:
# User function for converting the train and test dataset into spaCy document

def document(data) :
    #Creating empty list called "text"

    text = []
    for doc, label in nlp.pipe(data, as_tuples = True) :
        if(label == 'positive') :
            doc.cats['positive'] = 1
            doc.cats['negative'] = 0
            doc.cats['neutral'] = 0
        elif(label == 'negative') :
            doc.cats['positive'] = 0
            doc.cats['negative'] = 1
            doc.cats['neutral'] = 0
        else :
            doc.cats['positive'] = 0
            doc.cats['negative'] = 0
            doc.cats['neutral'] = 1
        
        #Adding the doc into the list 'text'
        text.append(doc)
    return(text)

In [10]:
#Calculate the time for converting into binary document for train dataset

start_time = datetime.now()

#passing the train dataset into function 'document'

train_docs = document(train)

#Creating binary document using DocBin function in spaCy

doc_bin = DocBin(docs = train_docs)

#Saving the binary document as "train.spacy" file
doc_bin.to_disk("train.spacy")
end_time = datetime.now()

#printing the time duration for train dataset
print('Duration : {}'.format(end_time - start_time))


Duration : 0:06:30.301209


In [11]:
#calculate the time for converting into binary document for test dataset

start_time = datetime.now()

#passing the test dataset into function 'document'
test_docs = document(test)
doc_bin = DocBin(docs = test_docs)
doc_bin.to_disk("valid.spacy")
end_time = datetime.now()

#printing the time duration for test dataset
print('Duration : {}'.format(end_time - start_time))

Duration : 0:01:52.989112


In [33]:
text = "australia's largest airline temporarily lays off 2500 employees"

#loading the best model from outputupdated folder
nlp = spacy.load("outputupdated/model-best")
demo = nlp(text)
print(demo.cats)

OSError: [E050] Can't find model 'outputupdated/model-best'. It doesn't seem to be a Python package or a valid path to a data directory.